In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

pd.options.display.max_rows = 999
pd.options.display.max_columns = None

# <font color = 'red'> Taking all columns - XGB </font>

### Base model

In [ ]:
target = 'is_female'
notuse_list = ['AA17', 'RI5_1','RI5_2','RI5_3','train_id', 'test_id']
predictors = [x for x in dtrain3.columns if x not in [target, notuse_list]]
xgb0 = XGBClassifier()
xgb0.fit(dtrain3[predictors], dtrain3[target])

In [ ]:
predictors_test =[x for x in dtest3.columns if x not in [target, notuse_list]]
xgb0_pred = xgb0.predict(dtest3[predictors_test])
xgb0_1 = pd.DataFrame()
xgb0_1['is_female'] = pd.Series(xgb0_pred).values
xgb0_1.to_csv("submission_xgb0.csv",header = ['is_female'], index = True, index_label = 'test_id')

In [ ]:
def modelfit_xgb(alg, dtrain, predictors, dtest, predictors_test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics=['auc'], early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target],eval_metric=['auc'])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain[target].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain[target], dtrain_predprob))
                    
#     feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')
    
    test_predprob = pd.DataFrame(alg.predict(dtest[predictors_test]))

    return test_predprob

In [ ]:
target = 'is_female'
notuse_list = ['AA17', 'RI5_1','RI5_2','RI5_3','train_id', 'test_id']
predictors = [x for x in dtrain3.columns if x not in [target, notuse_list]]
predictors_test =[x for x in dtest3.columns if x not in [target, notuse_list]]
xgb1 = XGBClassifier(learning_rate =0.1,
                     n_estimators=1000,
                     max_depth=5,
                     min_child_weight=1,
                     gamma=0,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     scale_pos_weight=1,
                     seed=27)
xgb1_ans = modelfit_xgb(xgb1, dtrain3, predictors, dtest3, predictors_test )

In [ ]:
xgb1_ans.to_csv("submission_xgb1.csv",header = ['is_female'], index = True, index_label = 'test_id')

# <font color = 'red'> Taking all columns - Random Forest </font>

In [ ]:
target = 'is_female'
notuse_list = ['AA17', 'RI5_1','RI5_2','RI5_3','train_id', 'test_id']
predictors = [x for x in dtrain3.columns if x not in [target, notuse_list]]
rf1 = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
rf1.fit(dtrain3[predictors], dtrain3[target])

In [ ]:
rf_ans = rf1.predict(dtest3[predictors_test])
rf_cm = confusion_matrix(dtest3[predictors_test], rf_ans)

rf_prob = rf1.predict_proba(dtest3[predictors_test])
metrics.roc_auc_score(dtest3[predictors_test], rf_prob[:,1])